# Notebook to analyze movement and determine thresholds

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load the landmarks data
landmarks_path = '../../data/input/landmarks/edited_villars_women_semifinals_n10_noplus_50_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

## Looking at Movement in the vertical direction

In [ ]:
# Define the keypoints for left and right hands and COM calculation
keypoints = {
    'left_wrist': 'left_wrist',
    'right_wrist': 'right_wrist',
    'left_hip': 'left_hip',
    'right_hip': 'right_hip',
    'left_shoulder': 'left_shoulder',
    'right_shoulder': 'right_shoulder'
}

# Extract wrist and COM keypoints (Y-coordinates) across frames
left_wrist_y = landmarks['left_wrist_y']
right_wrist_y = landmarks['right_wrist_y']

# Compute COM (Y-coordinate) as the average of shoulders and hips
COM_y = (landmarks['left_hip_y'] + landmarks['right_hip_y'] + landmarks['left_shoulder_y'] + landmarks['right_shoulder_y']) / 4

# Define the frames (time axis)
frames = landmarks.index

In [ ]:
# Plot the Y-coordinates of COM and the hand positions over the frames
plt.figure(figsize=(12, 8))
plt.plot(frames, COM_y, label='COM Y', color='blue', linestyle='solid')
plt.plot(frames, left_wrist_y, label='Left Wrist Y', color='red', linestyle='solid')
plt.plot(frames, right_wrist_y, label='Right Wrist Y', color='green', linestyle='solid')

plt.title('Vertical Movement (Y) of COM and Hands over Frames')
plt.xlabel('Frame')
plt.ylabel('Normalized Y-Coordinate')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Get the total number of frames
total_frames = len(landmarks)

print(f"Total number of frames: {total_frames}")

In [ ]:
# Define the keypoints for left and right hands and COM calculation
keypoints = {
    'left_wrist': 'left_wrist',
    'right_wrist': 'right_wrist',
    'left_hip': 'left_hip',
    'right_hip': 'right_hip',
    'left_shoulder': 'left_shoulder',
    'right_shoulder': 'right_shoulder'
}

# Extract wrist and COM keypoints (Y-coordinates) across frames
left_wrist_y = landmarks['left_wrist_y']
right_wrist_y = landmarks['right_wrist_y']

# Compute COM (Y-coordinate) as the average of shoulders and hips
COM_y = (landmarks['left_hip_y'] + landmarks['right_hip_y'] + landmarks['left_shoulder_y'] + landmarks['right_shoulder_y']) / 4

# Define the frames (time axis)
frames = landmarks.index

# Plot the Y-coordinates of COM and the hand positions over the frames
plt.figure(figsize=(12, 8))
plt.plot(frames, COM_y, label='COM', color='blue', linestyle='solid')
plt.plot(frames, left_wrist_y, label='Left Wrist', color='red', linestyle='solid')
plt.plot(frames, right_wrist_y, label='Right Wrist', color='green', linestyle='solid')

# Invert the Y-axis so that down means down
plt.gca().invert_yaxis()

# Set y-axis limits to include 0 and 1, even if data does not
plt.ylim(1.1,0)

# Set bigger font sizes for title, labels, and legend
plt.title('COM and Hands Movement in Gravitational Direction Over Frames', fontsize=20)
plt.xlabel('Frame', fontsize=16)
plt.ylabel('Normalized Y-Coordinate (Inverted)', fontsize=16)
plt.legend(fontsize=14)
plt.grid(True)

# Set bigger tick label size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.show()

### Looking at a specific sequence

In [ ]:
# Define the keypoints for left and right hands and COM calculation
keypoints = {
    'left_wrist': 'left_wrist',
    'right_wrist': 'right_wrist',
    'left_hip': 'left_hip',
    'right_hip': 'right_hip',
    'left_shoulder': 'left_shoulder',
    'right_shoulder': 'right_shoulder'
}

# Extract wrist and COM keypoints (Y-coordinates) across frames
left_wrist_y = landmarks['left_wrist_y']
right_wrist_y = landmarks['right_wrist_y']

# Compute COM (Y-coordinate) as the average of shoulders and hips
COM_y = (landmarks['left_hip_y'] + landmarks['right_hip_y'] + landmarks['left_shoulder_y'] + landmarks['right_shoulder_y']) / 4

# Define the frames (time axis)
frames = landmarks.index

# Plot the Y-coordinates of COM and the hand positions over the frames (zoomed in to frames 700-800)
plt.figure(figsize=(12, 8))
plt.plot(frames, COM_y, label='COM', color='blue', linestyle='solid')
plt.plot(frames, left_wrist_y, label='Left Wrist', color='red', linestyle='solid')
plt.plot(frames, right_wrist_y, label='Right Wrist', color='green', linestyle='solid')

# Invert the Y-axis so that down means down
plt.gca().invert_yaxis()

# Set x-axis limits to zoom in on the range of 700 to 800 frames
plt.xlim(800, 900)

# Set y-axis limits to include 0 and 1 with inverted axis
plt.ylim(1.1, 0)

# Set bigger font sizes for title, labels, and legend
plt.title('COM and Hands Movement in Gravitational Direction Over Frames', fontsize=20)
plt.xlabel('Frame', fontsize=16)
plt.ylabel('Normalized Y-Coordinate (Inverted)', fontsize=16)
plt.legend(fontsize=14)
plt.grid(True)

# Set bigger tick label size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.show()

## Looking at velocity

In [ ]:
# Load the landmarks data
landmarks_path = '../../data/input/landmarks/edited_villars_women_semifinals_n10_noplus_50_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

# Calculate the middle point of the hips (Y-coordinate)
hip_middle_y = (landmarks['left_hip_y'] + landmarks['right_hip_y']) / 2

# Define the frames (time axis)
frames = landmarks.index

# Compute velocity over a given number of frames
def calculate_velocity(y_coordinates, frames, step=10):
    velocities = []
    mid_frames = []

    for i in range(0, len(y_coordinates) - step, step):
        delta_y = y_coordinates[i + step] - y_coordinates[i]
        delta_t = frames[i + step] - frames[i]  # Assuming frames represent time
        velocity = delta_y / delta_t

        velocities.append(velocity)
        mid_frames.append((frames[i] + frames[i + step]) / 2)  # Midpoint frame for plotting

    return np.array(velocities), np.array(mid_frames)

# Function to plot the velocities with an optional zoom range
def plot_velocity(velocities, mid_frames, zoom_range=None):
    plt.figure(figsize=(12, 8))
    plt.plot(mid_frames, velocities, label='Hip Middle Point Velocity', color='purple', linestyle='solid')

    # Customize the plot
    plt.title('Velocity of Hip Middle Point Over Frames', fontsize=15)
    plt.xlabel('Frame', fontsize=16)
    plt.ylabel('Velocity (Normalized Y-Coordinate / Frame)', fontsize=16)
    plt.legend(fontsize=14)
    plt.grid(True)

    # Apply zoom if a range is provided
    if zoom_range:
        plt.xlim(zoom_range)
    
    # Set bigger tick label size
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    plt.show()

# Calculate velocities
velocities, mid_frames = calculate_velocity(hip_middle_y.values, frames, step=10)

# Default plot (no zoom)
plot_velocity(velocities, mid_frames)

# Zoomed-in plot (example zoom range: frames 700-800)
# plot_velocity(velocities, mid_frames, zoom_range=(700, 800))

## Eucledian Distance

In [ ]:
landmarks_path = '../../data/input/landmarks/edited_villars_women_semifinals_n10_noplus_50_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

# Extract x and y coordinates for the left and right hips
left_hip_x = landmarks['left_hip_x']
left_hip_y = landmarks['left_hip_y']
right_hip_x = landmarks['right_hip_x']
right_hip_y = landmarks['right_hip_y']

# Compute Euclidean distance over 10-frame intervals
def calculate_movement_over_interval(x_coords, y_coords, interval):
    distances = []
    for i in range(len(x_coords) - interval):
        x1, y1 = x_coords[i], y_coords[i]
        x2, y2 = x_coords[i + interval], y_coords[i + interval]
        distances.append(np.sqrt((x2 - x1)**2 + (y2 - y1)**2))
    return np.array(distances)

# Calculate 10-frame interval movements for both hips
interval = 10
left_hip_movement = calculate_movement_over_interval(left_hip_x, left_hip_y, interval)
right_hip_movement = calculate_movement_over_interval(right_hip_x, right_hip_y, interval)

# Define frames for plotting (accounting for the interval)
frames = np.arange(len(left_hip_movement))

# Plot the movements
plt.figure(figsize=(12, 8))
plt.plot(frames, left_hip_movement, label='Left Hip Movement', color='teal', linestyle='solid')
plt.plot(frames, right_hip_movement, label='Right Hip Movement', color='orange', linestyle='dotted')

# Customize the plot
plt.title('Movement of Left and Right Hips Over Frames', fontsize=15)
plt.xlabel('Frame', fontsize=16)
plt.ylabel('Euclidean Distance (10-Frame Interval)', fontsize=16)
plt.legend(fontsize=14)
plt.grid(True)

# Set bigger tick label size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.show()

In [ ]:
landmarks_path = '../../data/input/landmarks/edited_villars_women_semifinals_n10_noplus_50_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

# Extract x and y coordinates for the left and right hips
left_hip_x = landmarks['left_hip_x']
left_hip_y = landmarks['left_hip_y']
right_hip_x = landmarks['right_hip_x']
right_hip_y = landmarks['right_hip_y']

# Compute Euclidean distance over 10-frame intervals
def calculate_movement_over_interval(x_coords, y_coords, interval):
    distances = []
    for i in range(len(x_coords) - interval):
        x1, y1 = x_coords[i], y_coords[i]
        x2, y2 = x_coords[i + interval], y_coords[i + interval]
        distances.append(np.sqrt((x2 - x1)**2 + (y2 - y1)**2))
    return np.array(distances)

# Calculate 10-frame interval movements for both hips
interval = 10
left_hip_movement = calculate_movement_over_interval(left_hip_x, left_hip_y, interval)
right_hip_movement = calculate_movement_over_interval(right_hip_x, right_hip_y, interval)

# Define frames for plotting (accounting for the interval)
frames = np.arange(len(left_hip_movement))

# Define the zoomed range 
zoomed_in_frames = frames[800:900]
zoomed_in_left_hip_movement = left_hip_movement[800:900]
zoomed_in_right_hip_movement = right_hip_movement[800:900]

# Plot the movements (zoomed in)
plt.figure(figsize=(12, 8))
plt.plot(zoomed_in_frames, zoomed_in_left_hip_movement, label='Left Hip Movement', color='teal', linestyle='solid')
plt.plot(zoomed_in_frames, zoomed_in_right_hip_movement, label='Right Hip Movement', color='orange', linestyle='dotted')

# Customize the plot
plt.title('Movement of Left and Right Hip Over Frames 800 - 900', fontsize=15)
plt.xlabel('Frame', fontsize=16)
plt.ylabel('Euclidean Distance (10-Frame Interval)', fontsize=16)
plt.legend(fontsize=14)
plt.grid(True)

# Set bigger tick label size
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.show()

In [ ]:
# Path to the landmarks file
file_name = 'edited_lenzburg_final_women_cam16_n43_noplus_15'
# file_name = 'edited_villars_semifinals_men_cam4_n133_plus_19+'
input_video_path = f'../../data/input/landmarks/{file_name}_coordinates_local.parquet'

# Load the landmarks data
landmarks = pd.read_parquet(input_video_path)

# Extract x and y coordinates for the left and right hips
left_hip_x = landmarks['left_hip_x']
left_hip_y = landmarks['left_hip_y']
right_hip_x = landmarks['right_hip_x']
right_hip_y = landmarks['right_hip_y']

# Define frame dimensions (adjust as needed)
frame_width = 1920  # Example width in pixels
frame_height = 1080  # Example height in pixels

# Normalize coordinates
left_hip_x_normalized = (left_hip_x / frame_width) * 1000
left_hip_y_normalized = (left_hip_y / frame_height) * 1000
right_hip_x_normalized = (right_hip_x / frame_width) * 1000
right_hip_y_normalized = (right_hip_y / frame_height) * 1000

# Compute Euclidean distance over 10-frame intervals
def calculate_movement_over_interval(x_coords, y_coords, interval):
    distances = []
    for i in range(len(x_coords) - interval):
        x1, y1 = x_coords[i], y_coords[i]
        x2, y2 = x_coords[i + interval], y_coords[i + interval]
        distances.append(np.sqrt((x2 - x1)**2 + (y2 - y1)**2))
    return np.array(distances)

# Calculate 10-frame interval movements for both hips using normalized coordinates
interval = 10
left_hip_movement = calculate_movement_over_interval(left_hip_x_normalized, left_hip_y_normalized, interval)
right_hip_movement = calculate_movement_over_interval(right_hip_x_normalized, right_hip_y_normalized, interval)

# Define frames for plotting (accounting for the interval)
frames = np.arange(len(left_hip_movement))

# Plot the movements over the entire video
plt.figure(figsize=(12, 8))
plt.plot(frames, left_hip_movement, label='Left Hip Movement', color='teal', linestyle='solid')
plt.plot(frames, right_hip_movement, label='Right Hip Movement', color='orange', linestyle='dotted')

# Add a black vertical line at frame 830
#plt.axvline(x=830, color='black', linestyle='-', linewidth=2, label=None)
# plt.axvline(x=749, color='black', linestyle='-', linewidth=2, label=None)

# Customize the plot
plt.title('Movement of Left and Right Hip', fontsize=15)
plt.xlabel('Frame', fontsize=16)
plt.ylabel('Euclidean Distance (10-Frame Interval)', fontsize=16)
plt.legend(fontsize=14)
plt.grid(True)

# Set the y-axis range from 0.00 to 0.25
plt.ylim(0.00, 0.25)

# Set y-axis ticks with steps of 0.05
plt.yticks(np.arange(0.00, 0.26, 0.05))

# Add a horizontal line at y = 0.015
# plt.axhline(y=0.015, color='red', linestyle='--', label='Threshold')

plt.legend(fontsize=14, loc='upper left')

# Set bigger tick label size
plt.xticks(fontsize=14)

# Show the plot
plt.show()

In [ ]:
# Path to the landmarks file
# file_name = 'edited_lenzburg_final_women_cam16_n43_noplus_15'
file_name = 'edited_villars_semifinals_men_cam4_n133_plus_19+'
input_video_path = f'../../data/input/landmarks/{file_name}_coordinates_local.parquet'

# Load the landmarks data
landmarks = pd.read_parquet(input_video_path)

# Extract x and y coordinates for the left and right hips
left_hip_x = landmarks['left_hip_x']
left_hip_y = landmarks['left_hip_y']
right_hip_x = landmarks['right_hip_x']
right_hip_y = landmarks['right_hip_y']

# Define frame dimensions (adjust as needed)
frame_width = 1920  # Example width in pixels
frame_height = 1080  # Example height in pixels

# Normalize coordinates
left_hip_x_normalized = (left_hip_x / frame_width) * 1000
left_hip_y_normalized = (left_hip_y / frame_height) * 1000
right_hip_x_normalized = (right_hip_x / frame_width) * 1000
right_hip_y_normalized = (right_hip_y / frame_height) * 1000

# Compute Euclidean distance over 10-frame intervals
def calculate_movement_over_interval(x_coords, y_coords, interval):
    distances = []
    for i in range(len(x_coords) - interval):
        x1, y1 = x_coords[i], y_coords[i]
        x2, y2 = x_coords[i + interval], y_coords[i + interval]
        distances.append(np.sqrt((x2 - x1)**2 + (y2 - y1)**2))
    return np.array(distances)

# Calculate 10-frame interval movements for both hips using normalized coordinates
interval = 10
left_hip_movement = calculate_movement_over_interval(left_hip_x_normalized, left_hip_y_normalized, interval)
right_hip_movement = calculate_movement_over_interval(right_hip_x_normalized, right_hip_y_normalized, interval)

# Define frames for plotting (accounting for the interval)
frames = np.arange(len(left_hip_movement))


# Define the zoomed range n133
zoomed_in_frames = frames[749:828]
zoomed_in_left_hip_movement = left_hip_movement[749:828]
zoomed_in_right_hip_movement = right_hip_movement[749:828]

"""
# Define the zoomed range 
zoomed_in_frames = frames[830:878]
zoomed_in_left_hip_movement = left_hip_movement[830:878]
zoomed_in_right_hip_movement = right_hip_movement[830:878]
"""

# Plot the movements over the zoomed range
plt.figure(figsize=(12, 8))
plt.plot(zoomed_in_frames, zoomed_in_left_hip_movement, label='Left Hip Movement', color='teal', linestyle='solid')
plt.plot(zoomed_in_frames, zoomed_in_right_hip_movement, label='Right Hip Movement', color='orange', linestyle='dotted')

# Customize the plot
plt.title('Movement of Left and Right Hip (Interval of Interest)', fontsize=15)
plt.xlabel('Frame', fontsize=16)
plt.ylabel('Euclidean Distance (10-Frame Interval)', fontsize=16)
plt.legend(fontsize=14)
plt.grid(True)

# Set the y-axis range from 0.00 to 0.05
plt.ylim(0.00, 0.05)

# Add a horizontal line at y = 1.5e-05
# plt.axhline(y=1.5e-05, color='red', linestyle='--', label='Threshold')
plt.axhline(y=0.015, color='red', linestyle='--', label='Threshold')

# Set bigger tick label size
plt.xticks(fontsize=14)

# Add the legend in the top-left corner
plt.legend(fontsize=14, loc='upper left')

# Show the plot
plt.show()

## Looking at probability p

**visibility and presence:**
Visibility indicates the likelihood that a particular landmark is visible in the camera frame.
Presence indicates the likelihood that a particular landmark is present and tracked correctly.

In [ ]:
# Load the landmarks data
file_name = "edited_lenzburg_final_women_cam16_n52_plus_17+"
landmarks_path = f'../../data/input/landmarks/{file_name}_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

In [ ]:
print(landmarks)

In [ ]:
"""
# Extract p values for the left wrist, right wrist, left hip, and right hip
left_wrist_p = landmarks['left_wrist_p']
right_wrist_p = landmarks['right_wrist_p']
left_hip_p = landmarks['left_hip_p']
right_hip_p = landmarks['right_hip_p']

# Set display option to show all values
pd.set_option('display.max_rows', None)  # No row limit
pd.set_option('display.max_columns', None)  # No column limit
pd.set_option('display.width', None)  # No wrapping

# Now print the full list of p-values for left and right hips
print("\nFirst 100 Left Hip P Values:")
print(left_hip_p.head(100))

print("\nFirst 100 Right Hip P Values:")
print(right_hip_p.head(100))

# For the interval between frames 700-800
print("\nLeft Hip P Values from frame 700 to 800:")
print(left_hip_p.iloc[700:801])

print("\nRight Hip P Values from frame 700 to 800:")
print(right_hip_p.iloc[700:801])
"""

In [ ]:
# Load the landmarks data
file_name = "edited_lenzburg_final_women_cam16_n52_plus_17+"
landmarks_path = f'../../data/input/landmarks/{file_name}_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

# Presence indicates the likelihood that a particular landmark is present and tracked correctly. !!!

# Extract p values for the left wrist, right wrist, left hip, and right hip
left_wrist_p = landmarks['left_wrist_p']
right_wrist_p = landmarks['right_wrist_p']
left_hip_p = landmarks['left_hip_p']
right_hip_p = landmarks['right_hip_p']

# Define frames for plotting
frames = np.arange(len(left_wrist_p))

# Plot the p values
plt.figure(figsize=(14, 8))
plt.plot(frames, left_wrist_p, label='Left Wrist p', color='blue', linestyle='solid')
plt.plot(frames, right_wrist_p, label='Right Wrist p', color='green', linestyle='dotted')
plt.plot(frames, left_hip_p, label='Left Hip p', color='purple', linestyle='dashdot')
plt.plot(frames, right_hip_p, label='Right Hip p', color='orange', linestyle='dashed')

# Customize the plot
plt.title('P Values of Left Wrist, Right Wrist, Left Hip, and Right Hip', fontsize=16)
plt.xlabel('Frame', fontsize=14)
plt.ylabel('P Value', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)

# Show the plot
plt.legend(fontsize=12)
plt.show()

In [ ]:
# Load the landmarks data
file_name = "edited_lenzburg_final_women_cam16_n52_plus_17+"
landmarks_path = f'../../data/input/landmarks/{file_name}_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

# Extract p values for the left wrist, right wrist, left hip, and right hip
#left_wrist_p = landmarks['left_wrist_p'][:300]
#right_wrist_p = landmarks['right_wrist_p'][:300]
left_hip_p = landmarks['left_hip_p'][:300]
right_hip_p = landmarks['right_hip_p'][:300]

# Define frames for plotting
frames = np.arange(300)

# Plot the p values
plt.figure(figsize=(14, 8))
#plt.plot(frames, left_wrist_p, label='Left Wrist p', color='blue', linestyle='solid')
#plt.plot(frames, right_wrist_p, label='Right Wrist p', color='green', linestyle='dotted')
plt.plot(frames, left_hip_p, label='Left Hip p', color='purple', linestyle='dashdot')
plt.plot(frames, right_hip_p, label='Right Hip p', color='orange', linestyle='dashed')

# Customize the plot
plt.title('P Values of Left Hip, and Right Hip (Frames 1-300)', fontsize=16)
plt.xlabel('Frame', fontsize=14)
plt.ylabel('P Value', fontsize=14)
plt.legend(fontsize=12, loc='lower left')
plt.grid(True)

In [ ]:
# Load the landmarks data
file_name = "edited_lenzburg_final_women_cam16_n52_plus_17+"
landmarks_path = f'../../data/input/landmarks/{file_name}_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

# Extract v values for the left wrist, right wrist, left hip, and right hip
left_wrist_v = landmarks['left_wrist_v']
right_wrist_v = landmarks['right_wrist_v']
left_hip_v = landmarks['left_hip_v']
right_hip_v = landmarks['right_hip_v']

# Define frames for plotting
frames = np.arange(len(left_wrist_v))

# Plot the v values
plt.figure(figsize=(14, 8))
plt.plot(frames, left_wrist_v, label='Left Wrist v', color='blue', linestyle='solid')
plt.plot(frames, right_wrist_v, label='Right Wrist v', color='green', linestyle='dotted')
plt.plot(frames, left_hip_v, label='Left Hip v', color='purple', linestyle='dashdot')
plt.plot(frames, right_hip_v, label='Right Hip v', color='orange', linestyle='dashed')

# Customize the plot
plt.title('V Values of Left Wrist, Right Wrist, Left Hip, and Right Hip', fontsize=16)
plt.xlabel('Frame', fontsize=14)
plt.ylabel('V Value', fontsize=14)
plt.legend(fontsize=12, loc='lower left')
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Load the landmarks data
file_name = "edited_lenzburg_final_women_cam16_n52_plus_17+"
landmarks_path = f'../../data/input/landmarks/{file_name}_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

# Extract v values for the left wrist, right wrist, left hip, and right hip
left_wrist_v = landmarks['left_wrist_v'][:300]
right_wrist_v = landmarks['right_wrist_v'][:300]
left_hip_v = landmarks['left_hip_v'][:300]
right_hip_v = landmarks['right_hip_v'][:300]

# Define frames for plotting
frames = np.arange(300)

# Plot the v values
plt.figure(figsize=(14, 8))
plt.plot(frames, left_wrist_v, label='Left Wrist v', color='blue', linestyle='solid')
plt.plot(frames, right_wrist_v, label='Right Wrist v', color='green', linestyle='dotted')
plt.plot(frames, left_hip_v, label='Left Hip v', color='purple', linestyle='dashdot')
plt.plot(frames, right_hip_v, label='Right Hip v', color='orange', linestyle='dashed')

# Customize the plot
plt.title('V Values of Left Wrist, Right Wrist, Left Hip, and Right Hip (Frames 1-300)', fontsize=16)
plt.xlabel('Frame', fontsize=14)
plt.ylabel('V Value', fontsize=14)
plt.legend(fontsize=12, loc='lower left')
plt.grid(True)

# Show the plot
plt.show()

In [ ]:
# Parameters
distance_threshold = 0.15
interval = 10  # Interval over which to calculate movement

# Load the landmarks data
file_name = "edited_lenzburg_final_women_cam16_n44_plus_17+"
landmarks_path = f'../../data/input/landmarks/{file_name}_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

# Extract hip coordinates
left_hip_x = landmarks['left_hip_x']
left_hip_y = landmarks['left_hip_y']
right_hip_x = landmarks['right_hip_x']
right_hip_y = landmarks['right_hip_y']

# Calculate the interval-based movement of individual hips
left_hip_movement = np.sqrt(
    (left_hip_x[interval:].values - left_hip_x[:-interval].values) ** 2 +
    (left_hip_y[interval:].values - left_hip_y[:-interval].values) ** 2
)
right_hip_movement = np.sqrt(
    (right_hip_x[interval:].values - right_hip_x[:-interval].values) ** 2 +
    (right_hip_y[interval:].values - right_hip_y[:-interval].values) ** 2
)

# Adjust frame indices to match intervals
frame_indices = np.arange(interval, len(left_hip_x))  # Start from the first complete interval

# Plot the movements
plt.figure(figsize=(12, 6))
plt.plot(frame_indices, left_hip_movement, label="Left Hip Movement (Interval)", color="blue")
plt.plot(frame_indices, right_hip_movement, label="Right Hip Movement (Interval)", color="green")

# Add the threshold line
plt.axhline(y=distance_threshold, color="red", linestyle="--", label="Distance Threshold")

# Add labels and legend
plt.title(f"Hip Movement Over {interval}-Frame Intervals")
plt.xlabel("Frame Index")
plt.ylabel("Euclidean Distance (Over Interval)")
plt.legend()
plt.grid()

# Show the plot
plt.show()

In [ ]:
# Parameters
distance_threshold = 0.15
interval = 10  # Interval over which to calculate movement

# Load the landmarks data
file_name = "edited_villars_semifinals_men_cam14_n114_plus_35+"
landmarks_path = f'../../data/input/landmarks/{file_name}_coordinates_local.parquet'
landmarks = pd.read_parquet(landmarks_path)

# Extract hip coordinates
left_hip_x = landmarks['left_hip_x']
left_hip_y = landmarks['left_hip_y']
right_hip_x = landmarks['right_hip_x']
right_hip_y = landmarks['right_hip_y']

# Calculate the interval-based movement of individual hips
left_hip_movement = np.sqrt(
    (left_hip_x[interval:].values - left_hip_x[:-interval].values) ** 2 +
    (left_hip_y[interval:].values - left_hip_y[:-interval].values) ** 2
)
right_hip_movement = np.sqrt(
    (right_hip_x[interval:].values - right_hip_x[:-interval].values) ** 2 +
    (right_hip_y[interval:].values - right_hip_y[:-interval].values) ** 2
)

# Adjust frame indices to match intervals
frame_indices = np.arange(interval, len(left_hip_x))  # Start from the first complete interval

# Plot the movements
plt.figure(figsize=(12, 6))
plt.plot(frame_indices, left_hip_movement, label="Left Hip Movement (Interval)", color="blue")
plt.plot(frame_indices, right_hip_movement, label="Right Hip Movement (Interval)", color="green")

# Add the threshold line
plt.axhline(y=distance_threshold, color="red", linestyle="--", label="Distance Threshold")

# Add labels and legend
plt.title(f"Hip Movement Over {interval}-Frame Intervals")
plt.xlabel("Frame Index")
plt.ylabel("Euclidean Distance (Over Interval)")
plt.legend()
plt.grid()

# Show the plot
plt.show()